In [112]:
import yfinance as yf
import numpy as np
from typing import List
import pandas as pd

Vector = List[float]

desvios_utilizados = 1.7 # tendencia a partir de x desvios padrões
media_utilizada = 10 # dias


In [113]:
def media_x(x: int, v: Vector) -> Vector:
  return ["n.a." if i < x-1 else np.mean(v[i-x+1:i+1]) for i in range(len(v))]

def desv_x(x: int, v: Vector) -> Vector:
  return ["n.a." if i < x-1 else np.std(v[i-x+1:i+1]) for i in range(len(v))]

def tendencia(desvios: float, preço: Vector, mediax: Vector, desvx: Vector) -> Vector:
  tend = {}
  for i in range(len(preço)):
    if mediax[i] == "n.a.":
      tend[i] = "n.a."
    elif preço[i] > mediax[i] + desvios * desvx[i]:
      tend[i] = "up"
    elif preço[i] < mediax[i] - desvios * desvx[i]:
      tend[i] = "down"
    else:
      tend[i] = tend[i-1]
  return list(tend.values())


In [114]:
BOVX11_infos = yf.Ticker("BOVX11.SA")
BOVX11_cotação = BOVX11_infos.history(period="1y")["Close"]

In [115]:
BOVX = pd.DataFrame()

BOVX["Preço"] = BOVX11_cotação
BOVX["Média 10 dias"] = media_x(media_utilizada, BOVX11_cotação)
BOVX["Desvio 10 dias"] = desv_x(media_utilizada, BOVX11_cotação)
BOVX["Tendência"] = tendencia(desvios_utilizados, BOVX["Preço"], BOVX["Média 10 dias"], BOVX["Desvio 10 dias"])